In [1]:
!pip install datasets -q
!pip install tokenizers -q
!pip install transformers==4.28.0 
!pip install seqeval -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


In [4]:
import pandas as pd
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import torch

## Introduction

Hello,
First I would like to apologize for any shortcomings in completing your coding challenge. I had found the corresponding email late and had to put in a lot of code in a very short time.

In [93]:
df = pd.read_csv("/kaggle/input/drug-master-list/Drugs Master List.csv")
df.head(10)

,drug_name,medical_condition,side_effects,generic_name,drug_classes,brand_names,activity,rx_otc,pregnancy_category,csa,alcohol,related_drugs,medical_condition_description,rating,no_of_reviews,drug_link,medical_condition_url
0,doxycycline,Acne,"(hives, difficult breathing, swelling in your ...",doxycycline,"Miscellaneous antimalarials, Tetracyclines","Acticlate, Adoxa CK, Adoxa Pak, Adoxa TT, Alod...",87%,Rx,D,N,X,amoxicillin: https://www.drugs.com/amoxicillin...,Acne Other names: Acne Vulgaris; Blackheads; B...,6.8,760.0,https://www.drugs.com/doxycycline.html,https://www.drugs.com/condition/acne.html
1,spironolactone,Acne,hives ; difficulty breathing; swelling of your...,spironolactone,"Aldosterone receptor antagonists, Potassium-sp...","Aldactone, CaroSpir",82%,Rx,C,N,X,amlodipine: https://www.drugs.com/amlodipine.h...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.2,449.0,https://www.drugs.com/spironolactone.html,https://www.drugs.com/condition/acne.html
2,minocycline,Acne,"skin rash, fever, swollen glands, flu-like sym...",minocycline,Tetracyclines,"Dynacin, Minocin, Minolira, Solodyn, Ximino, V...",48%,Rx,D,N,NaN,amoxicillin: https://www.drugs.com/amoxicillin...,Acne Other names: Acne Vulgaris; Blackheads; B...,5.7,482.0,https://www.drugs.com/minocycline.html,https://www.drugs.com/condition/acne.html
3,Accutane,Acne,problems with your vision or hearing; muscle o...,isotretinoin (oral),"Miscellaneous antineoplastics, Miscellaneous u...",NaN,41%,Rx,X,N,X,doxycycline: https://www.drugs.com/doxycycline...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.9,623.0,https://www.drugs.com/accutane.html,https://www.drugs.com/condition/acne.html
4,clindamycin,Acne,hives ; difficult breathing; swelling of your ...,clindamycin topical,"Topical acne agents, Vaginal anti-infectives","Cleocin T, Clindacin ETZ, Clindacin P, Clindag...",39%,Rx,B,N,NaN,doxycycline: https://www.drugs.com/doxycycline...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.4,146.0,https://www.drugs.com/mtm/clindamycin-topical....,https://www.drugs.com/condition/acne.html
5,Aldactone,Acne,hives ; difficulty breathing; swelling of your...,spironolactone,"Aldosterone receptor antagonists, Potassium-sp...",CaroSpir,35%,Rx,C,N,X,amlodipine: https://www.drugs.com/amlodipine.h...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.6,8.0,https://www.drugs.com/aldactone.html,https://www.drugs.com/condition/acne.html
6,tretinoin,Acne,hives ; difficult breathing; swelling of your ...,tretinoin topical,Topical acne agents,"Altreno, Atralin, Avita, Refissa, Renova, Reti...",30%,Rx,C,N,NaN,NaN,Acne Other names: Acne Vulgaris; Blackheads; B...,7.7,439.0,https://www.drugs.com/mtm/tretinoin-topical.html,https://www.drugs.com/condition/acne.html
7,isotretinoin,Acne,problems with your vision or hearing; muscle o...,isotretinoin (oral),"Miscellaneous antineoplastics, Miscellaneous u...","Absorica, Accutane, Amnesteem, Claravis, Myori...",26%,Rx,X,N,X,doxycycline: https://www.drugs.com/doxycycline...,Acne Other names: Acne Vulgaris; Blackheads; B...,8.0,999.0,https://www.drugs.com/mtm/isotretinoin.html,https://www.drugs.com/condition/acne.html
8,Bactrim,Acne,"skin rash, fever, swollen glands, joint pain, ...",sulfamethoxazole and trimethoprim,Sulfonamides,Bactrim DS,20%,Rx,D,N,X,NaN,Acne Other names: Acne Vulgaris; Blackheads; B...,8.5,96.0,https://www.drugs.com/bactrim.html,https://www.drugs.com/condition/acne.html
9,Retin-A,Acne,hives; difficult breathing; swelling of your f...,Retin-A,Topical acne agents,NaN,17%,Rx,C,N,NaN,NaN,Acne Other names: Acne Vulgaris; Blackheads; B...,7.9,86.0,https://www.drugs.com/retin-a.html,https://www.drugs.com/condition/acne.html


There are 999 samples with 17 features and all features are in string format.

In [96]:
print("Column names : " , df.columns)
print("Dataframe shape: ", df.shape)

Column names :  Index(['drug_name', 'medical_condition', 'side_effects', 'generic_name',
       'drug_classes', 'brand_names', 'activity', 'rx_otc',
       'pregnancy_category', 'csa', 'alcohol', 'related_drugs',
       'medical_condition_description', 'rating', 'no_of_reviews', 'drug_link',
       'medical_condition_url'],
      dtype='object')
Dataframe shape:  (999, 17)


## Data Exploration
After observing the dataset, I noticed several problems with the dataset:
- drugs have inappropriate medical conditions e.g spironolactone is a diuretic and not used for acne
- There were inconsistencies in how the content of each feature sample were given (different formats)
- Some features are irrelevant for the task.
- some features contain null values.

In [98]:
df["drug_classes"].value_counts()

Upper respiratory combinations                                        172
Topical acne agents                                                    88
Antiviral combinations                                                 47
CNS stimulants                                                         39
Atypical antipsychotics                                                26
                                                                     ... 
Miscellaneous antidepressants                                           1
Nutraceutical products                                                  1
Antirheumatics, TNF alfa inhibitors                                     1
Antihistamines, Miscellaneous anxiolytics, sedatives and hypnotics      1
Phenothiazine antipsychotics                                            1
Name: drug_classes, Length: 117, dtype: int64

In [99]:
df.isnull().sum()

drug_name                          0
medical_condition                  0
side_effects                      28
generic_name                      12
drug_classes                      21
brand_names                      386
activity                           0
rx_otc                             0
pregnancy_category                72
csa                                0
alcohol                          495
related_drugs                    508
medical_condition_description      0
rating                           492
no_of_reviews                    492
drug_link                          0
medical_condition_url              0
dtype: int64

After much observation, we will be working with only the features below.

In [100]:
relevant_features = ['drug_name', 'medical_condition', 'side_effects', 'generic_name',
       'drug_classes', 'brand_names', 'pregnancy_category', 'csa', 'related_drugs','drug_link']

df = df[relevant_features]

## Data Cleaning Solution
1. To correct all the discrepancies in the data, we will simply do this:
- we will use scraping functions to fetch details about each drug name on the following features: medical_condtion, generic_name,brand_name. If their corresponding values cannot be found, we will use their default values.
- For the pregnacy_category, I simply used the Mode value to fill nan values as I could not find a good website containing this feature for all the drugs in the dataset.
- For the side effects feature, most drugs have a side effect of "nausea, vomiting +/- headaches". Therefore, I use that as the default value to fill out nan values here.
- For the drug_class, I use the mode value for that feature.

I use beautiful soup and requests library below.


In [101]:
import requests
from bs4 import BeautifulSoup

#Function fetches the 
def get_medical_condition(row):
  drug_name = row["drug_name"].lower().split()[0]
  url = f"https://www.drugs.com/dosage/{drug_name}.html"
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")
  if soup.find('div', class_='conditionTree'):
    condition_list = soup.find("div", class_="conditionTree").find("ul").text.strip("\n").replace("\n", ", ").split(",")
    condition_list = ", ".join(condition_list)
    return condition_list
  else:
    return row["medical_condition"]
  
def get_generic_name(row):
  if pd.isnull(row['generic_name']):
    return row['drug_name']
  else:
    return row['generic_name']

#TODO
def get_brand_names(row):
  if pd.isnull(row["brand_names"]) :
    #print('yes')
    drug_name = row["drug_name"].lower().split(" ")[0]
    #print(drug_name)
    url = f"https://www.merckmanuals.com/home/SearchResults?query={drug_name}"
    #print(url)
    page = requests.get(url)

    soup = BeautifulSoup(page.content, "html.parser")
    #print(soup.find("article", class_="search__result"))
    #print(soup.find('p', class_="search__result--shortinfo"))
    if soup.find("article", class_="search__result").find('div', class_="search__result--shortinfo"):
      condition_list = soup.find("div", class_="search__result--shortinfo").text.replace('Brand Name(s)', '').strip("\n").split(",")
      condition_list = ",".join(condition_list)
      #print("GET: ", condition_list)
      return condition_list
    else:
      #print(drug_name)
      return drug_name
  else:
    return row["brand_names"]
    
def get_drug_class(row):
  if pd.isnull(row["drug_classes"]):
    drug_name = row["drug_name"].lower().split(" ")[0]
    #print(drug_name)
    url = f"https://www.drugguide.com/ddo//search?st=OSS&q={drug_name}"
    page = requests.get(url)

    soup = BeautifulSoup(page.content, "html.parser")
    if soup.find('ul', class_="resultList"):
      link = soup.find("ul", class_="resultList").find("li").find("ul").find("li").find("a").get("href")
      link = "https://www.drugguide.com/ddo/"+ link
      #print(link)
      page = requests.get(link)
      
      drug_classes = ""
      soup = BeautifulSoup(page.content, "html.parser")
      if soup.find("section", class_="section") != None :
        #print(soup.find("section", class_="section"))
        class_links = soup.find("section", class_="section").find_all("p")
        
        for ind, each in enumerate(class_links):
          if 'Pronunciation' not in each.text and "Ther. Class."  in each.text:
            if len(class_links) > ind + 2 :
              drug_classes += class_links[ind + 2].text 
            if len(class_links) > ind + 3:
              drug_classes += ", " + class_links[ind +3].text
          if 'Pronunciation' not in each.text and "Pharm. Class." in each.text:
            drug_classes = drug_classes + ", " + class_links[ind+2].text
        drug_classes = drug_classes.rstrip(",")
        drug_classes = drug_classes.replace("Pharm. Class.", "")

        #print(drug_classes, " : " + drug_name)
      if drug_classes == "":
        return "Upper respiratory combinations"
          
      return drug_classes
    else:
      return "Upper respiratory combinations"
  else:
    return row['drug_classes']
  
def get_side_effects(row):
  if pd.isnull(row["side_effects"]):
    return "headaches, nausea, vomiting"
  else:
    return row["side_effects"]

def get_pregnancy_category(row):
  if pd.isnull(row["pregnancy_category"]):
    return "C"
  else:
    return row["pregnancy_category"]


def get_related_drugs(row):
  drug_name = row["drug_name"].lower().split(" ")[0]
  #print(drug_name)
  url = f"https://www.drugguide.com/ddo//search?st=OSS&q={drug_name}"
  page = requests.get(url)

  soup = BeautifulSoup(page.content, "html.parser")
  if soup.find('ul', class_="resultList"):
    link = soup.find("ul", class_="resultList").find("li").find("ul").find("li").find("a").get("href")
    link = "https://www.drugguide.com/ddo/"+ link
    #print(link)
    page = requests.get(link)
    
    soup = BeautifulSoup(page.content, "html.parser")
    related_links = soup.find_all("div", class_="sidebox-list-item")
    if related_links:
      related_drugs = ""

      for i in related_links:
        #print(i.text)
        related_drugs +=  i.text + "' "
      return related_drugs

    else:
      return drug_name
  else:
    return drug_name



In [ ]:
df["related_drugs"] = df.apply(get_related_drugs, axis=1)


In [ ]:
df["medical_condition"] = df.apply(get_medical_condition, axis=1)
df["side_effects"] = df.apply(get_side_effects ,axis=1)
df["generic_name"] = df.apply(get_generic_name, axis=1)
df["drug_classes"] = df.apply(get_drug_class, axis= 1)
df["pregnancy_category"] = df.apply(get_pregnancy_category, axis=1)
df["brand_names"] = df.apply(get_brand_names, axis=1)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

We further clean the data here removing any unwanted symbols or punctuations or extra spacings.

In [ ]:
def clean_data(row):
  effects = row["side_effects"].lower().replace("warning", "").replace("caution", "").replace("&","").replace(":", "").replace(";",",")
  row["side_effects"] = effects.replace("(", "").replace(")", "").replace("Uses","").replace("Warnings", "").replace("Before taking", "").replace("Side effects", "").replace("Interactions","").replace("indications","")
  row["brand_names"] = row["brand_names"].replace("Uses","").replace("Warnings", "").replace("Before taking", "").replace("Side effects", "").replace("Interactions","").replace("indications","")
  #row["medical_condition"] = row["medical_condition"].replace("[","").replace("]","")
  row["related_drugs"] = row["related_drugs"].replace("†","").replace("'", ",")
  return row


df_new= df.apply(clean_data, axis=1)
df_new.head(10)

I save the cleaned data to a local csv file.

In [ ]:
df.to_csv("/content/drive/MyDrive/AI study/drug_data.csv")

In [14]:
df = pd.read_csv("/kaggle/input/drug-list/drug_data.csv")

## Problem Statement
Now, we are done cleaning and processing the data. How do we get the model this problem?
What are the possible queries that can be given by a user?

Possible cases with queries
- Just drug names are provided to the IR system e.g: amoxicillin
- full sentences/phrases can be provided (e.g "I want information on spironolactone", "Get doxycycline")
- Very complex queries could be provided.
- drug names might be mispelled : "parcetamol, doxycylin, amoxicilllin"
- drug abbreviations might be used : "PCM for paracetamol, diclo for diclofenac, IBP for ibuprofen "
- generic names or drug classes may be given instead of the drug names
- The drug specified might not be in the database.

Additionally, in this case, this retrieval system will definitely differ from others because most times clinicians or health workers want information for a particlar drug and not other similar drugs.

Therefore, we need a robust system to handle most of these problems. To do this, our system will need to recognize and extract medically named entities (e.g drug, side effect) and it has to be robust to spelling errors. 

We could use sophisticated large language models like BERT or ADA to run cosine or other similarity metrics on embeddings of the user's queries but that comes with a lot of problems intuitively. I mention some of them here:

- We might need to finetune this model as a lot of models have not been really trained on pharmaceutical data, however, our pharmaceutical data is small.  word embedding models are trained model but most do not really process pharmaceuticals word embedding is a trained model but most do not really process pharmaceuticals

- how does the model interprete wrongly spelt drugs? What does word embedding truly meanfor the model here? does it offer any semantics? word embedding models are trained using the words they are mostly associated with in a particular dataset. word embedding models are trained using the words they are mostly associated with in a particular dataset.

- What happens when their several drugs in a complex user query? e.g "I want information on paracetamol , orphenadrine alongside amlodipine. I need information on their side effects."

- What about drug abbreviations? how does it handle that? What if a drug is actually a phrase ? e.g paracetamol injection or efavirenz/ tenofovir?I mean word embedding is a trained model but most do not really process pharmaceuticals. What about drug abbreviations? how does it handle that? What if a drug is actually a phrase ? e.g paracetamol injection or efavirenz/ tenofovir?

In this case, that is difficult to model. Feeding " I want information on paracetamol" might give you a good embedding but how do we store drug embedding of the drug "paracetamol" without any context in our database? It might end up being a very expensive or inefficient process storing vector embeddings.
Is the word "paracetamol" just passed as input for conversion to an embedding or do we pass some context to it? In any of these cases, the retrieved results would be affected.

To this effect, I focus instead on making an Named Entity Recognition + rule-based system to tackle this problems. With this model, we eliminate the use of vector dbs, hosting large models with large requirements online or paying for ada gpt subscriptions while just identifying drugs alongside some other identifiable drug attributes.

My first step is to create an artificial dataset using manually crafted template (in the context of clinical settings and queries as that is where the solution will be utilized) that contains thousands of data using different drug names, medical conditions, side effects, related drug, generic names etc from our original dataset. 

We use this data to finetune a BERT model for the task of NER. I will add random noise and spelling errors to the medical entities most importantly so that the model is more robust.


Note: Drug abbreviations can also be added to this data set generation to enable the model recognize abbreviations like PCM, IBP etc. Also, we can use a pretrained auto correction model or train one to recognize and correct spelling errors. Due to time constraint, I skip these steps.

## Dataset Creation/Generation
I manually wrote about 75 sentences,that will be used to generate our artificial data. Each sample was written in the context of a clinical worker prying for details about drugs.
I created placeholders within these texts that will be substituted with different categories in our data.

For example, "drug_name" will be replaced with a random drug in our dataset, "generic_name" will be replaced with a generic drug in our data and so on.

The function create_dataset() iterates through each drug sample in our database. Features of a drug sample in our database will be used to replace the place holder of a particular proportion of these 75 sentences (75% used here) randomly.

Also, a function called add_noise will be used to add noise to any particular drug attribute. For example, the drug "diclofenac" can become dicl#fenac or diclonac etc.
The create_dataset function makes use of the add_noise and data_generator function which is the core of the data generation step.

I also prompted gpt for more but I was unable to add those due to time constraints. The 75 sentences are found below.

In [15]:
data_template = [
  "If you look into drug_link, you will get information about drug_name including its side effects. It is a very common medication used to treat medical_conditions .",
  "The medication drug_name or popularly / commonly known as generic_name belongs to class pregnancy_category and can be found in many brands including brand_names .",
  "Examples of drug_class are drug_name, brand_name, generic_name .",
  "Examples of medications that contain generic_name are brand_name .",
  "Examples of medicatioins / drugs that are drug_class are brand_name and generic_name . But they have side_effects as side effects. ",      
  "Check out this related_drug at drug_link .",  
  "You can find more about the indications , dosage, dose , side effects, interactions of drug_name or generic_name at drug_link . Don't forget to check other drugs in its drug_class .",
  "What are the commone side effects of drug_name? Tell me if the medical conditions for it include medical_conditions .",
  "What are the other drugs that belong to the drug_class family .",
  "What is the generic for drug_name along side the generic name of related_drug ?",
  "Tell me some of the drugs that can be found in the drug class drug_class .",
  "I want to prescribe drug_name for a patient. Can you tell me the drugs that could be associated with such ?",
  "I want to get more information about the drug drug_name and its related drug related_drug .",
  "What are the uses of drug_name medication along side its side effects and interactions .",
  "I have forgotten the class of the drug drug_name. Can you get it for me? ",
  "I can't remember the class of drug_name. Tell me .",
  "what class of drug is the drug drug_name ?",
  "what class of drug is drug_name and related_drug .",
  "I can't remember the class that drug drug_name belongs to. Can you look it up for more information ?",
  "These side effects side_effects are commonly found when you use drugs that belong to drug_class .",
  "These side effects side_effects are commonly found when you use drugs that belong to drug class drug_class .",
  "These side effects side_effects are commonly found when you use drugs that belong to drug_name , related_drug .",
  "drug_name can be used to treat medical_conditions .",
  "The drug drug_name has a side effect of causing side_effect symptoms. That is why it falls under category pregnancy_category .",
  "Find more information about drug_name and generic_name and brand_name at drug_link .",
  "What is similar to drug_name ? It is related_drug with side effects side_effects .",
  "The following brand_name all have the same component which is drug_name or also known as generic_name .",
  "The following brand all have the same component which is drug_name or also known as generic_name .",
  "The active constituent of the brand name brand_name is drug_name also known as generic_name",
  "The brand name of drug_name is/are brand_name but you can also find them as generic_name. They are very useful in treating medical_conditions .",
  "The following branded drugs or brand names - brand_name all have the same component which is drug_name or also known as generic_name ."
  "The active ingredient in brand_name is drug_name also called as generic_name that belongs to the family drug_class .",
  "The active ingredient in brand_name brand is drug_name also called as generic_name that belongs to the family drug_class .",
  "The active ingredient in brand_name is drug_name and it belongs to the family drug_class used to treat medical_conditions .",
  "The active ingredient in brand_name is drug_name and it belongs to the family drug_class used to treat illnesses .",
  "Get drug_name", "drug_class", "drug_name", "generic_name .", "medical_conditions", "drug", "drug class", "brand name", 
  "drug_name and related related_drug belong to the family drug_class .",
  "These drugs: drug_name , related_drug probably have the same side effect which is side_effects .",
  "The active ingredient in brand_name is drug_name also known as generic_name .",
  "medical_conditions can be treated using drug_name with these side effects: side_effects .",
  "Side effect of drug drug_name is side_effects ."
  "medical_conditions can be treated using drug_name .",
  "The following medical_conditions can be treated with drug_name but you have to be worried about side_effects .",
  "The following medical_conditions can be treated with drug but you have to be worried about the following side effects: side_effects .",
  "Give me information about drug_name .",
  "Give me information about the drug drug_name .",
  "The side effects of drug_name are side_effects .",
  "The side effects of the drug drug_name are side_effects .",
  "The side effects of drug_name drug are  side_effects . ",
  "The drug drug_name belongs to the drug_class and is used to treat medical conditions like medical_conditions .",
  "The drug drug_name belongs to the drug_class and is used to treat illness like medical_conditions .",
  "The drug drug_name belongs to the drug_class and is used to treat illnesses like medical_conditions .",
  "The drug drug_name belongs to the drug_class and is used to treat ailments like medical_conditions . it is recommended by the FDA for such.",
  "drug_name belongs to the drug_class and is used to treat medical conditions like medical_conditions .",
  "drug_name is used to treat medical conditions like medical_conditions .",
  "drug_name is used to treat medical_conditions medical conditions .",
  "drug_name is used to treat medical_conditions medical conditions .",
  "drug_name is a drug_class and is related to related_drug and useful in treating medical_conditions .",
  "You can find more information about drug_name at drug_link ",
  "You can find more information about the drug drug_name with the generic_name at drug_link .",
  "I want information on drug_name .",
  "I want information on drug drug_name .",
  "I want information on drug_class .",
  "I want information on related_drug .",
  "I want information on generic_name .",
  "I want examples of drug_class .",
  "Show me drugs that are similar to drug_name .",
  "Show me medications that have similar side effects as drug_name .",
  "what drug can be used to treat medical_contitions ?",
  "I want information on drug drug_name with brand brand_name with side_effects side effects, generic name generic_name used to treate medical condition\
  medical_conditions with a drug class of drug_class and related to related_drug. it has drug_link and pregnancy_category and dosage of 10 dose ."
  ]

Additionally, I create some important variables that will be used by model during inference. We have to save the important drug categories, information categories that exist and can be pulled up by our retrieval script.

In [16]:
DRUG_CATEGORY = ['B-GENERIC', 'B-DRUGNAME', 'I-BRAND', 'I-GENERIC', 'B-BRAND', 'I-DRUGNAME']
DRUG_FEATURES = ["drug_name","brand_name", "generic_name", "related_drugs"]

INFORMATION_CATEGORY = [ 'I-DOSE', 'I-SYMPTOMS', 'B-ILL', 'B-BRANDNAME', 
                        'I-BRANDNAME',  'I-ILL_NAME', 'B-ILL_NAME', 'I-DRUGCLASS',
                        'B-GENERICNAME', 'I-CLASS', 'I-SIDE', 'I-GENERICNAME','B-CLASS',
                        'I-ILL', 'B-SYMPTOMS', 'O', 'B-DRUG', 'B-SIDE', 
                        'B-DRUGCLASS']

TAGS_TO_FEATURES = {'B-GENERIC': "generic name", 'B-DRUGNAME': ["related_drugs","drug_name"],
'I-DOSE': ["dose","dosage", "dosages"], 'I-SYMPTOMS':"symptoms", 'B-ILL':"medical_condition",  'B-BRANDNAME':"brand_names",  
'I-BRANDNAME': "brand_names", 'I-BRAND': "drug brand", 'I-ILL_NAME':"medical condition", 
'B-ILL_NAME':"medical condition", 'I-DRUGCLASS' : "drug class", 'B-GENERICNAME': "generic_name", 
'I-GENERIC':"generic name", 'B-BRAND':"drug band", 'I-CLASS':"drug_class",
'I-SIDE':"side_effects", 'I-GENERICNAME': "generic_name", 'I-DRUGNAME': ["related_drugs","drug_name"], 
'B-CLASS':"drug_class", 'I-ILL':"medical_condition", 'B-SYMPTOMS':"symptoms", 'O': "others",
'B-DRUG': "drug", 'B-SIDE': "side_effects", 'B-DRUGCLASS': "drug class"}

The group of functions below is what is used to generate the artificial dataset like I have explained before. The above is just some boiler plate code to generate data so I won't go into it.

In [17]:
import random
import string  as st

characters = st.ascii_letters + st.digits + st.punctuation


def to_lower(string,weights = [0.5,0.5]):
    if random.choices([True,False],weights=weights ):
        return string.lower()
    else:
        return string
    
def add_noise(string, add_noise_prob=0.15, noise_prob=0.1 ):
  insert_noise = random.choices([0,1], weights=[1 - add_noise_prob, add_noise_prob])[0]
  if not insert_noise:
    return string
  else:
    # Add noise : insert new character or remove character:
    remove_character = random.choice([0,1])
    if remove_character:
      n_characters = round(noise_prob * len(string))
      char_indices = random.sample(list(range(len(string))), n_characters )
      edited_string = ""
      for pos, i in enumerate(string):
        if pos not in char_indices:
          edited_string += i
      #return edited_string
    else:
      n_characters = round(noise_prob * len(string))
      new_characters = random.sample(characters, n_characters)
      positions = random.sample(range(len(string)), n_characters)
      edited_string = list(string)
      for c , p in zip(new_characters, positions):
        edited_string.insert(p, c)
      edited_string  = "".join(edited_string)

    return edited_string


def data_generator(row, perc=0.75, add_noise_prob=0.1, noise_prob=0.1):
  data_samples  = []
  ner_tags = []
  #print(len(data_template))
  n_samples = round(perc * len(data_template))
  #print("Number of samples: ", n_samples)
  template = random.sample(data_template, n_samples)
  
    each_split = each.lower().split(" ")
    ner_tag = [ "O" for i in range(len(each_split))]
   
    for index, word in enumerate(each_split):
      if word == "drug_class":
        classes = row["drug_classes"].strip(" ").replace("/", " / ").split(", ")
        if len(classes) > 1:
          classes = random.sample(classes, random.choice(range(1,len(classes))))
        temp = []
        tags = []
        for ind, class_ in enumerate(classes):
          #print("class:", class_)
          if class_ not in "/,":
            for i , each in enumerate(class_.split(" ")):
              #print("each: ", each)
              temp.append(add_noise(to_lower(each), add_noise_prob, noise_prob))
              if i == 0:
                tags.append('B-DRUGCLASS')
              else:
                tags.append("I-DRUGCLASS")          
            temp.append(",")
            tags.append("O")
          else:
            temp.append(class_)
            tags.append("O")
           

        each_split[index] = temp[0]
        ner_tag[index] = tags[0]

        # print(len(temp), len(tags))
        for pos in range(1, len(tags)):
          each_split.insert(index + pos, temp[pos])
          ner_tag.insert(index+pos, tags[pos]) 

        continue 

      if word == "pregnancy_category":
        each_split[index] = row["pregnancy_category"]
        continue

      if word == "related_drug":
        #each_split[index] = row["related_drugs"]
        related_drugs = row["related_drugs"].replace("/", " / ").split(", ")
        if len(related_drugs) > 1:
          related_drugs = random.sample(related_drugs, random.choice(range(1,len(related_drugs))))
        temp = []
        tags = []
        for ind, drug in enumerate(related_drugs):
          if drug not in "/,":
            for i , each in enumerate(drug.split(" ")):
              temp.append(add_noise(to_lower(each), add_noise_prob, noise_prob))
              if i == 0:
                tags.append('B-DRUGNAME')
              else:
                tags.append("I-DRUGNAME")          
            #related_drugs.insert(ind + 1, ',')
            temp.append(",")
            tags.append("O")
          else:
            temp.append(drug)
            tags.append("O")

        each_split[index] = temp[0]
        ner_tag[index] = tags[0]

        for pos in range(1, len(tags)):
          each_split.insert(index + pos, temp[pos])
          ner_tag.insert(index+pos, tags[pos]) 
        continue

      if word == "drug_link" :
        each_split[index]= row["drug_link"]
        continue

      if word == "brand_name":
        drugs = row["brand_names"].strip(" ").replace("/", " / ").split(",")
        if len(drugs) > 1:
          drugs = random.sample(drugs, random.choice(range(1,len(drugs))))
        temp = []
        tags = []
        for ind, drug in enumerate(drugs):
          if drug not in "/,":
            for i , d in enumerate(drug.strip(" ").split(" ")):
              temp.append(add_noise(to_lower(d), add_noise_prob, noise_prob))
              #temp.append(d)
              if i == 0:
                tags.append('B-BRAND')
              else:
                tags.append("I-BRAND")
            temp.append(",")
            tags.append("O")
            #drugs.insert(ind + 1, ",")
            #tags.append("O")
          else:
            temp.append(drug)
            tags.append("O")

        # print("temp brand names:", temp)
        # print(temp[0])
        each_split[index] = temp[0]
        ner_tag[index] = tags[0]
        for pos in range(1, len(tags)):
          #print(drugs[pos])
          each_split.insert(index + pos, temp[pos])
          ner_tag.insert(index+pos, tags[pos])

        continue  
      

      if word == "drug_name":
        drugs = row["drug_name"].strip(" ").replace("/", " / ").replace(",", " ").split(" ")
        temp =[]
        tags = []
        for ind, drug in enumerate(drugs):
          if drug != "/":
            temp.append(add_noise(to_lower(drug), add_noise_prob, noise_prob))

            if ind== 0:
              tags.append('B-DRUGNAME')
            else:
              tags.append("I-DRUGNAME")
          else:
            temp.append(drug)
            tags.append("O")

        
        each_split[index] = temp[0]
        ner_tag[index] = tags[0]

        for pos in range(len(tags)):
          each_split.insert(index + pos, temp[pos])
          ner_tag.insert(index+pos, tags[pos]) 
        continue   

      if word == "side_effects":
        effects = row["side_effects"].strip(" ").replace("/", " / ").split(",")
        if len(effects) > 1:
          effects = random.sample(effects, random.choice(range(1,len(effects))))
        temp = []
        tags = []
        for ind, effect in enumerate(effects):
          for i, eff in enumerate(effect.strip(" ").split(" ")):
            temp.append(add_noise(to_lower(eff), add_noise_prob, noise_prob))
            #temp.append(eff)
            if eff not in ",:/":
              if i== 0:
                tags.append('B-SYMPTOMS')
              else:
                tags.append("I-SYMPTOMS")
            else:
              tags.append("O")
          temp.append(",")
          tags.append("O")
          
        #print(len(temp), len(tags))
        # print("temp side effect:", temp)
        # print(temp[0])
        each_split[index] = temp[0]
        ner_tag[index] = tags[0]

        for pos in range(1, len(tags)):
          each_split.insert(index + pos, temp[pos])
          ner_tag.insert(index+pos, tags[pos]) 

        continue

      if word == "generic_name":
        drugs = row["generic_name"].strip(" ").replace("/", " / ").split(",")
        if len(drugs) > 1:
          drugs = random.sample(drugs, random.choice(range(1,len(drugs))))
        temp = []
        tags = []
        for ind, drug in enumerate(drugs):
          if drug not in "/,:;":
            for i , each in enumerate(drug.strip(" ").split(" ")):
              temp.append(add_noise(to_lower(each), add_noise_prob, noise_prob))
              #temp.append(each)
              if i == 0:
                tags.append('B-GENERIC')
              else:
                tags.append("I-GENERIC")          
            temp.append(",")
            tags.append("O")
          else:
            temp.append(drug)
            tags.append("O")

        #print(len(temp), len(tags))
        each_split[index] = temp[0]
        ner_tag[index] = tags[0]
        for pos in range(1, len(tags)):
          each_split.insert(index + pos, temp[pos])
          ner_tag.insert(index+pos, tags[pos]) 
    
        
        continue

      if word == "medical_conditions":
        illnesses = row["medical_condition"].strip(" ").replace("/", " / ").split(", ")
        if len(illnesses) > 1:
          illnesses = random.sample(illnesses, random.choice(range(1, len(illnesses))))
        temp = []
        tags = []
        for ind, illness in enumerate(illnesses):
          if illness not in "/,":
            for i , each in enumerate(illness.strip(" ").split(" ")):
              temp.append(add_noise(to_lower(each), add_noise_prob, noise_prob))
              #temp.append(each)
              if i == 0:
                tags.append('B-ILL_NAME')
              else:
                tags.append("I-ILL_NAME")          
            temp.append(",")
            tags.append("O")
          else:
            temp.append(illness)
            tags.append("O")

        #print(len(temp), len(tags))
        each_split[index] = temp[0]
        ner_tag[index] = tags[0]
        for pos in range(1, len(tags)):
          each_split.insert(index + pos, temp[pos])
          ner_tag.insert(index+pos, tags[pos]) 
    
        continue

     
      if word == "class" and ner_tag[index] == "O":
        ner_tag[index] = "B-CLASS"
        continue 

      if index + 1 < len(each_split):
        if index + 2 < len(each_split):
          if (word == "class" and each_split[index + 1] == "of" and ( each_split[index + 2] == "drug" or each_split[index + 2]=="drugs")) and ner_tag[index] == "O":
            ner_tag[index] = "B-CLASS"
            ner_tag[index + 1] = "I-CLASS"
            ner_tag[index + 2] = "I-CLASS"
            continue

        if (word == "side" and (each_split[index + 1] == "effects" or each_split[index + 1]=="effect")) and ner_tag[index] == "O":
          ner_tag[index] = "B-SIDE"
          ner_tag[index + 1] = "I-SIDE"
          continue

        if (word == "drug" and (each_split[index + 1] == "name" or each_split[index + 1] == "names")) and ner_tag[index] == "O":
          ner_tag[index] = "B-DRUG"
          ner_tag[index + 1] = "I-DRUG"
          continue

        if (word == "brand" and (each_split[index + 1] == "name" or each_split[index + 1] == "names")) and ner_tag[index] == "O":
          ner_tag[index] = "B-BRANDNAME"
          ner_tag[index + 1] = "I-BRANDNAME"
          continue

        if (word == "drug" and (each_split[index + 1] == "class" or each_split[index + 1] == "classes")) and ner_tag[index] == "O":
          ner_tag[index] = "B-CLASS"
          ner_tag[index + 1] = "I-CLASS"
          continue

        if (word == "generic" and (each_split[index + 1] == "name" or each_split[index + 1] == "names")) and ner_tag[index] == "O":
          ner_tag[index] = "B-GENERICNAME"
          ner_tag[index + 1] = "I-GENERICNAME"
          continue

        if word == "medical" and (each_split[index + 1] == "conditions" or each_split[index + 1]=="condition"):
          ner_tag[index] = "B-ILL"
          ner_tag[index] = "I-ILL"
          continue

      if (word == "illness" or word == "illnesses" or word == "ailment" or word == "ailments")and ner_tag[index] == "O":
        ner_tag[index] = "B-ILL"
        continue

      if word == "brand" and ner_tag[index] == "O":
        ner_tag[index] = "B-BRANDNAME"
        continue

      if word == "generic" and ner_tag[index] == "O":
        ner_tag[index] = "B-GENERICNAME"
        continue

      if (word == "drug" or word == "medication" or word == "medications" or word == "drugs") and ner_tag[index] == "O": 
        ner_tag[index] = "B-DRUG"
        continue

      if (word == "dose" or word == "dosage" or word == "dosages") and ner_tag[index] == "O":
        ner_tag[index] = "I-DOSE"
        continue

    data_samples.append(each_split)
    ner_tags.append(ner_tag)

    # print(len(data_samples))
    # print(len(ner_tags))

  return data_samples, ner_tags


Now, we create the function to rule them all. This function creates the dataset for us using the functions above


In [18]:
def create_dataset(df, perc_template=0.75, add_noise_prob=0.12, noise_prob=0.1):
  data_samples = []
  ner_tags = []

  for i in range(len(df)):
    #print(i)
    samples, tags =  data_generator(df.iloc[i], perc_template, add_noise_prob, noise_prob)
    data_samples.extend(samples)
    ner_tags.extend(tags)

  return data_samples , ner_tags


In [19]:
data_samples, ner_tags = create_dataset(df)

In [20]:
len(data_samples) , len(ner_tags)

(55944, 55944)

55944 samples were created. That is pretty impressive.

In [21]:
print(data_samples[0]), print(ner_tags[0])

['check', 'out', 'this', 'amobarbital', ',', 'ferric', 'citrate', ',', 'pentobarbital', ',', 'asUcorbic', 'acid', ',', '', 'at', 'https://www.drugs.com/doxycycline.html', '.']
['O', 'O', 'O', 'B-DRUGNAME', 'O', 'B-DRUGNAME', 'I-DRUGNAME', 'O', 'B-DRUGNAME', 'O', 'B-DRUGNAME', 'I-DRUGNAME', 'O', 'O', 'O', 'O', 'O']


(None, None)

The cell below generates the unique labels e.g "B-DRUG" in the generated data and maps them to unique integers that will be used during training.

After this, we generate the tags present in each data sample using the convert_labels() method.

In [22]:
label_names = list(set([element for each_sample in ner_tags for element in each_sample]))
label_index = {tag : index for index, tag in enumerate(label_names)}
index_label  = {value: key for key, value in label_index.items()}

def convert_labels():
  data_tags = []
  for sample in ner_tags:
    temp = []
    for each in sample:
      temp.append(label_index[each])
    data_tags.append(temp)

  return data_tags


tags = convert_labels()
#print(data_tags[0])

In [50]:
from datasets import load_dataset, Dataset


In [52]:
data = Dataset.from_dict({"tokens": data_samples, "ner_tags": tags})
data

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 55944
})

Split the generated data into train , test and validation sets.

In [53]:
if "train" not in data.column_names:
        data = data.train_test_split(
            test_size= 0.2
        )
        data_test_valid = data["test"].train_test_split(
            test_size=0.5
        )
        data["test"] = data_test_valid["train"]
        data["validation"] = data_test_valid["test"]

train_dataset = data["train"]
eval_dataset = data["validation"]
test_dataset = data["test"]


Now, we have divided the generated data into train , test and validation, let's save our data in a parquet file. But first, we convert every drug in the table to lower case.

In [54]:
def to_lower(row):
    row["drug_name"] = row["drug_name"].lower()
    row["generic_name"] = row["generic_name"].lower()
    row["brand_names"] = row["brand_names"].lower()
    row["related_drugs"] = row["related_drugs"].lower()
    return row

df_copy = df.apply(to_lower, axis=1)
df_copy.head()

,Unnamed: 0,drug_name,medical_condition,side_effects,generic_name,drug_classes,brand_names,pregnancy_category,csa,related_drugs,drug_link
0,0,doxycycline,"Acne, Actinomycosis, Amebiasis, Chancroid, ...","hives, difficult breathing, swelling in your f...",doxycycline,"Miscellaneous antimalarials, Tetracyclines","acticlate, adoxa ck, adoxa pak, adoxa tt, alod...",D,N,"ferric maltol, ferric citrate, sucroferric oxy...",https://www.drugs.com/doxycycline.html
1,1,spironolactone,"Edema, Hypertension, Congestive Heart Failur...","hives , difficulty breathing, swelling of your...",spironolactone,"Aldosterone receptor antagonists, Potassium-sp...","aldactone, carospir",C,N,"aldactazide 25/25, aldactazide 50/50, cosyntro...",https://www.drugs.com/spironolactone.html
2,2,minocycline,"Bacterial Infection, Acne, Meningitis - Meni...","skin rash, fever, swollen glands, flu-like sym...",minocycline,Tetracyclines,"dynacin, minocin, minolira, solodyn, ximino, v...",D,N,"isotretinoin, vitamin b complex with vitamin c...",https://www.drugs.com/minocycline.html
3,3,accutane,Acne,"problems with your vision or hearing, muscle o...",isotretinoin (oral),"Miscellaneous antineoplastics, Miscellaneous u...",accutane,X,N,"sarecycline, doxycycline, minocycline, tetracy...",https://www.drugs.com/accutane.html
4,4,clindamycin,Bacterial Infection,"hives , difficult breathing, swelling of your ...",clindamycin topical,"Topical acne agents, Vaginal anti-infectives","cleocin t, clindacin etz, clindacin p, clindag...",B,N,"veltin topical gel, ziana topical gel, tretino...",https://www.drugs.com/mtm/clindamycin-topical....


Now, let's remove the "unnamed: 0" feature, set "drug_name" as index for faster retrievals and save data in parquet file.

In [55]:
df_copy = df.reset_index(drop=True)
df_copy = df_copy.iloc[:,1:]
df_copy.set_index("drug_name", inplace=True)
df_copy= df_copy.astype(str)
df_copy.to_parquet("drugs_info.parquet", engine="pyarrow", compression='gzip', index=True)

For the task of NER, we use the "distilbert-base-cased" model and tokenizer. Its a 260mb model that can be upload anywhere for inference. Therefore this model really saves cost in terms of hardware requirement, simplicity in production and deployment!

It can also run inference on cpu with a lower latency as compared to other large language models. We will see below that this model acquires an average performance of about 91% accuracy on this task of NER after training for just 6 epochs!

Not only that, this model was able to identify other queried attributes for any given drug and not just rely on the provided focus! For example, It was able to identify side effects , medical conditions associated with drugs and knew when it was prompted for things like pregnancy category etc.

Note: Previous model was lost and the currently model didnt train up to 6 epochs due to time constraints on my part.

In [56]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["tokens"], padding="max_length", truncation=True, is_split_into_words=True)

#tokenized_datasets_ = data.map(tokenize_function, batched=True)



Now that we have a function that tokenizes the data, this function however does not ensure that the tokenized words and corresponding tags have equal lengths.

For example the sentence: "The black dog is thirsty" -> "the", "bl" ,"##ack","dog", "is","th","##irsty" will still have the tags "article", "adjective", "noun", "verb", "adjective".

We can see from this example, that the length of the tokenized subword is not equal to that of the tags.

The function below helps to correct this problem.

In [57]:
#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_adjust_labels(all_samples_per_split):
  tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], truncation=True, is_split_into_words=True,)
  #tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used 
  #so the new keys [input_ids, labels (after adjustment)]
  #can be added to the datasets dict for each train test validation split
  total_adjusted_labels = []
  #print(len(tokenized_samples["input_ids"]))
  for k in range(0, len(tokenized_samples["input_ids"])):
    prev_wid = -1
    word_ids_list = tokenized_samples.word_ids(batch_index=k)
    existing_label_ids = all_samples_per_split["ner_tags"][k]
    #print(existing_label_ids)
    i = -1
    adjusted_label_ids = []
   
    for wid in word_ids_list:
      if(wid is None):
        adjusted_label_ids.append(-100)
      elif(wid!=prev_wid):
        i = i + 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = wid
      else:
        adjusted_label_ids.append(existing_label_ids[i])
        
    total_adjusted_labels.append(adjusted_label_ids)
  tokenized_samples["labels"] = total_adjusted_labels
  return tokenized_samples



Now, let's use the above function to tokenize the train, test and eval data.

In [58]:
train_dataset = train_dataset.map(tokenize_adjust_labels, batched=True, remove_columns=["tokens", "ner_tags"], )
eval_dataset = eval_dataset.map(tokenize_adjust_labels, batched=True, remove_columns=["tokens", "ner_tags"], )
test_dataset = test_dataset.map(tokenize_adjust_labels, batched=True, remove_columns=["tokens", "ner_tags"], )


  0%|          | 0/45 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

we need to add pad tokens so all samples are of the same length as some samples would naturally have longer sentences than others. We use the DataCollatorForTokenClassifion here.


In [59]:
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer
import numpy as np
from datasets import load_metric
metric = load_metric("seqeval")

data_collator = DataCollatorForTokenClassification(tokenizer, padding='max_length')

We load the model using the AutoModelForTokenClassification class which applies the right heads to any given model for the task of NER.

In [60]:
model = AutoModelForTokenClassification.from_pretrained("distilbert-base-cased", num_labels=len(label_names))

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

In [61]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
      if(k not in flattened_results.keys()):
        flattened_results[k+"_f1"]=results[k]["f1"]

    return flattened_results


Set batch size, logging steps and number of epochs

In [70]:
batch_size = 16
logging_steps = len(train_dataset) // batch_size
epochs = 4

We define the model hyperparameter, data and training arguments to be used by the model for training below.

In [71]:
training_args = TrainingArguments(
    output_dir="./fine_tune_bert_output",
    evaluation_strategy="steps",
    #eval_steps = 10,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.01,
    logging_steps = 50,
    save_strategy='steps', 
    save_steps = 100,
    save_total_limit = 1,
    disable_tqdm = False, 
    gradient_accumulation_steps = 64,
    #eval_accumulation_steps = 4,
    lr_scheduler_type = 'cosine',
    warmup_steps = 20,
    fp16 = True,
    #load_best_model_at_end = True,
    #metric_for_best_model = "loss",
    optim = "adamw_torch",
    report_to = "tensorboard"

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_dataset,
    eval_dataset= eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


We use the trainer API to commence training of the model

In [125]:
trainer.train()

Step,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,Brand F1,Brandname F1,Class F1,Dose F1,Drug F1,Drugclass F1,Drugname F1,Generic F1,Genericname F1,Ill F1,Ill Name F1,Side F1,Symptoms F1
50,0.194800,0.169104,0.893884,0.879346,0.886555,0.929708,0.974079,0.979048,0.931217,0.862434,0.974891,0.924010,0.921830,0.919159,0.808314,0.931276,0.902609,0.891803,0.762791
100,0.170400,0.158254,0.900043,0.886078,0.893006,0.933719,0.973581,0.979981,0.933687,0.875332,0.976143,0.926526,0.924190,0.924687,0.812065,0.934266,0.916220,0.876221,0.780658
150,0.163500,0.156292,0.901691,0.882418,0.891950,0.934030,0.973651,0.979981,0.932540,0.873016,0.975770,0.927370,0.924684,0.922457,0.810185,0.934327,0.912152,0.885371,0.776849


TrainOutput(global_step=172, training_loss=0.1743186074633931, metrics={'train_runtime': 4862.627, 'train_samples_per_second': 36.815, 'train_steps_per_second': 0.035, 'total_flos': 2.3016141432966144e+16, 'train_loss': 0.1743186074633931, 'epoch': 3.93})

Now, Let's test this finetuned model on our test dataset.

In [126]:
trainer.evaluate()
trainer.save_model('/kaggle/working/finetuned_NER.model')

predictions, labels, _ = trainer.predict(test_dataset)

predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)

true_predictions = [

[label_names[p] for (p, l) in zip(prediction, label) if l != -100]

for prediction, label in zip(predictions, labels)

]

true_labels = [

[label_names[l] for (p, l) in zip(prediction, label) if l != -100]

for prediction, label in zip(predictions, labels)

]

results = metric.compute(predictions=true_predictions, references=true_labels)

results = pd.DataFrame(results)
results.head()

,BRAND,BRANDNAME,CLASS,DOSE,DRUG,DRUGCLASS,DRUGNAME,GENERIC,GENERICNAME,ILL,ILL_NAME,SIDE,SYMPTOMS,overall_precision,overall_recall,overall_f1,overall_accuracy
precision,0.965074,1.000000,0.935705,0.810256,0.988406,0.928899,0.921395,0.934534,0.893491,0.975485,0.916667,0.900524,0.787228,0.894782,0.882922,0.888812,0.934129
recall,0.957031,0.982422,0.921833,0.872928,0.962597,0.932789,0.924657,0.949718,0.712264,0.890028,0.911435,0.857428,0.751537,0.894782,0.882922,0.888812,0.934129
f1,0.961036,0.991133,0.928717,0.840426,0.975331,0.930840,0.923023,0.942065,0.792651,0.930799,0.914043,0.878447,0.768969,0.894782,0.882922,0.888812,0.934129
number,7680.000000,512.000000,742.000000,181.000000,2834.000000,4776.000000,35584.000000,5847.000000,212.000000,1073.000000,5081.000000,1003.000000,23424.000000,0.894782,0.882922,0.888812,0.934129


## Model Output Postprocessing
Having trained our models, there is still a lot of post-processing to be done to get this model's result useful for query to our database. The following must be done:
- Since subword tokenization is used by the model's tokenizer, we must aggregate subwords together again to form words before it is fed to the model. However, we must also aggregate the predicted tags per subword accordingly.

The question then is, since we have to aggregate tags too, how do we choose the right tag for a particular word (after aggregation)? For example the word doxycycline may be broken down into 3 subwords:
"do", "##xy", "##cycline" with corresponding tags "B-DRUGNAME", "B-BRAND", "B-DRUGNAME". For this, we write a function (aggregate) that either selects the appropriate tag using the mode (i.e "B-DRUGNAME" in this example) or using the tag with the highest probability (for example, "B-BRAND" is selected out of the 3 if it has the highest probability).

- Secondly, after aggregation of words how do we precess compound drug name? For example, assuming the drug "Ipratropium bromide" is a compound name with eventual tags "B-DRUGNAME", "I-DRUGNAME". I create  a function ("join_words_tags") that handles this. It moves from right to left looking for beginning tags (i.e "B-"). Once its found, it checks the next occuring terms if the have the same tail (i.e DRUGNAME in this case) and start with an "I". This function is called by the aggregate function in the code.

After these 2 steps ,the processed results is then fed to the retrieve_result() function that looks up the retrieved drug names in the parquet database.

In [127]:
def join_words_tags(wp):
    """
        Function aggregates words together based on the predicted tags.
    """
    subword =False
    # Create a temporary empty list to store words, predictions, scores.
    temp_word = []
    temp_pred = []
    temp_score = []
    
    # Create lists to store eventual processsed words, tags and corresponding probability score.
    input_words = []
    tags = []
    prob_scores = []

    tag_processed = [False for i in range(len(wp["ner"]))]
    
    # Iterate through each words in the list
    for ind,(word, tag, score) in enumerate(zip(wp["words"],wp["ner"], wp["prob. score"])):
        if not tag_processed[ind] and tag.startswith("B-"):
          
            subword = True                                      
            temp_word.append(word)
            #temp_pred.append(pred)
            temp_score.append(score)
            tag_processed[ind] = True
            
            if ind + 1 >= len(wp["ner"]):
                input_words.append(word)
                tags.append(tag)
                prob_scores.append(score)
                subword = False
                break
            else:
                while subword == True:

                    for i in range(ind+1, len(wp["ner"])):
                        if wp["ner"][i].startswith("I-") and wp['ner'][i][2:] == tag[2:]:
                            temp_word.append(wp["words"][i])
                            temp_score.append(wp["prob. score"][i])
                            tag_processed[i] = True
                        else:
                            input_words.append(" ".join(temp_word))
                            tags.append(tag)
                            prob_scores.append(np.mean(temp_score))
                            temp_word = []
                            temp_pred = []
                            temp_score = []    

                            subword = False
                            break
            
            continue
         
        if not tag_processed[ind] and tag.startswith("I-"):
            
            input_words.append(word)
            tags.append(tag)
            prob_scores.append(score)
            tag_processed[ind] = True
            

        elif not tag_processed[ind] and tag.startswith("O"):
            
            input_words.append(word)
            tags.append(tag)
            prob_scores.append(score)
            tag_processed[ind] = True
            continue
            
        else:
            continue
            

    return {"ner": tags, "words": input_words, 'prob. score': prob_scores}


def aggregate(words,predictions,prob_score,use_max= True):
   
    assert (len(words) == len(predictions))and (len(predictions) == len(prob_score))
    prob_score.pop(0)
    prob_score.pop(-1)
    reversed_prob_score = prob_score[::-1]
 
    words.pop(0)
    words.pop(-1)
    reversed_words= words[::-1]
    
    predictions.pop(0)
    predictions.pop(-1)    
    reversed_predictions = predictions[::-1]
           

    input_words = []
    tags = []
    prob_scores = []

    subword =False
    temp_word = []
    temp_pred = []
    temp_score = []

    for ind ,(word, pred, score) in enumerate(zip(reversed_words, reversed_predictions, reversed_prob_score)):
        if word.startswith("##"):
            subword = True                                      
            temp_word.insert(0, word.replace("##",""))
            temp_pred.insert(0, pred)
            temp_score.insert(0, score)
        else:
            if subword:
                temp_word.insert(0, word)
                temp_pred.insert(0, pred)
                temp_score.insert(0,score)
                if use_max:                    
                    input_words.insert(0,"".join(temp_word))
                    tags.insert(0, temp_pred[np.argmax(temp_score)])
                    prob_scores.insert(0, np.max(temp_score))
                else:
                    input_words.insert(0,"".join(temp_word))
                    if len(temp_pred) <= 2:
                        tags.insert(0, temp_pred[np.argmax(temp_score)])
                        prob_scores.insert(0, np.max(temp_score))
                    else:
                        modals, counts = np.unique(temp_pred, return_counts=True)
                        index = np.argmax(counts)
                        tags.insert(0, modals[index])
                        sum_pred = []
                        for ind, t in enumerate(temp_pred):
                            if t == modals[index]:
                                sum_pred.append(temp_score[ind])
                        prob_scores.insert(0, np.mean(sum_pred))
                    
                temp_word=[]
                temp_pred=[]
                temp_score=[]
                subword = False
            else:
                input_words.insert(0, word)
                tags.insert(0, pred)
                prob_scores.insert(0, score)
                #subword =False
                
    words_prediction  = join_words_tags({"ner": tags, "words": input_words, 'prob. score': prob_scores})

    return words_prediction




I test it on an example here, We see that the join_word_tags function does fine. The aggregate function will be tested later in the notebook.

In [128]:
words = ["Paracetamol", "Orphenadrine", "Diclofenac", "or" ,"ibuprofen", "is", "good", "for", "headaches",
        "vomiting","nausea","Amlodipine","hives"]
t  = ["B-DRUGNAME", "I-DRUGNAME","I-DRUGNAME", "O", "I-DRUGNAME", "O","O", "O", "B-ILL", "I-SIDE", "I-SIDE",
     "B-DRUGNAME","I-SIDE"]
score = [0.67,0.5, 0.22, 0.45, 0.674, 0.91, 0.82,0.21, 0.89,0.4,0.10,0.99,0.78]

print(len(words), len(t), len(score))
pd.DataFrame(join_words_tags({"words": words, "ner": t, "prob. score": score}))[:]

13 13 13


,ner,words,prob. score
0,B-DRUGNAME,Paracetamol Orphenadrine Diclofenac,0.463333
1,O,or,0.450000
2,I-DRUGNAME,ibuprofen,0.674000
3,O,is,0.910000
4,O,good,0.820000
5,O,for,0.210000
6,B-ILL,headaches,0.890000
7,I-SIDE,vomiting,0.400000
8,I-SIDE,nausea,0.100000
9,B-DRUGNAME,Amlodipine,0.990000


In [129]:
len(label_names)

25

Lastly, we define functions below that will:
- Load the model and tokenizer (load_model_tokenizer())
- Get_response from the model when a query is given to it (get_response())
- Post process predictions of models using the aggregate function (post_process())
- Lastly, retrieve all identifiable results using the extracted named entities recognized.

The get_response() function is the main function that handles everything.

An extra functionality was added to 

In [130]:
def load_model_tokenizer(path, num_labels):
    # Load trained model and tokenizer
    tokenizer =  AutoTokenizer.from_pretrained(path)
    model =  AutoModelForTokenClassification.from_pretrained(path, num_labels=num_labels)
    return model , tokenizer


def get_response(model, tokenizer, query, parquet_path ,focus=None, threshold=0.1, use_max=False):
    """
        Retrieves drug information from parquet database using the model's predicted output.
    """
    # Tokenize query
    tokens = tokenizer(query)      
    # Fetch predictions. Shape == (1, n_subwords, len(label_names))
    predictions = model.forward(input_ids=torch.tensor(tokens['input_ids']).unsqueeze(0),
                    attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))
    # Get the probability score for each predictions
    prob_score = list(torch.max(torch.softmax(predictions.logits, axis=-1).squeeze(),
                                axis=1).values.detach().numpy())
    #Get the maximum predicted values from these predictions.
    # Shape = (1, n_subwords, 1)
    predictions = torch.argmax(predictions.logits.squeeze(), axis=1)

    # Get tokens ids
    input_ids = torch.tensor(tokens["input_ids"])
#     pred_for_word = [label_names[i] for i in predictions]
#     each_word = tokenizer.batch_decode(input_ids)
    # Post process the generated outputs and their input_ids using the tokenizer.    
    words_prediction = post_process(tokenizer , input_ids, predictions, prob_score, use_max)
    
    # Retrieve results from the parquet database using predicted entity tags.
    results = retrieve_results(words_prediction,parquet_path, focus, threshold=threshold)
    
    return results #words_prediction #

def post_process(tokenizer, input_ids, pred, prob_score,use_max=False):
    
    """
        This function aggregates the word ids and corresponding tags into full words and then into 
        compound words if necessary based on the generated entity tag.
    """
    # Change all generated label to theirtags (e.g 1  -> "B-DRUG")
    pred_for_word = [label_names[i] for i in pred]
    # Tokenize input ids to words (e.g 1 -> "##xy")
    each_word = tokenizer.batch_decode(input_ids)
    # Aggregate subwords into full words and then into compound words based on tag predictions.
    words_tags = aggregate(each_word, pred_for_word, prob_score, use_max)
        
    return words_tags

def retrieve_results(words_prediction,path, focus=None,threshold=0.7):
    """
        Function retrieves results from the parquet database and provide the results.
    
    """
    #  Create result object
    results = {"results": []}
    # create list to store extracted drug names
    drugs = []    
    # Create list to store extracted attributes
    if not focus:
        focus = []
     
   # Loop through each word and corresponding predicted tags 
    for each_word, pred, score in zip(words_prediction['words'], words_prediction["ner"], words_prediction["prob. score"]):
        # if the model's predicted tag is in the drug_category append to the drug list
        if pred in DRUG_CATEGORY and score >= threshold:
            drugs.append(each_word)   
        # Else append to features / attribute list
        elif pred in INFORMATION_CATEGORY and score >= threshold:
            feat = TAGS_TO_FEATURES[pred]
            
            if type(feat)== list:
                focus.extend(feat)
            else:
                focus.append(feat)   
        # If the model does not predict a drugname or drug feature for a particular word, ignore it.
        else:
            continue
            
    drugs = set(drugs)
    focus = set(focus)
   
    
    # if drug list is empty, return an empty dictionary
    if len(drugs) == 0:
        return {"results": None}
        
#     elif len(drugs) > 1 and len(focus) == 0:
#         return {"results" : None}

#     elif len(drugs) == 0 and len(focus) > 0:
#         return {"results" : None}

    # Open database:
    db = pd.read_parquet(path)

    # Iterate through each found drug.
    for drug in drugs:
        drug = drug.lower()
        drug_info_found = False
        # if the drug is in the index
        if drug in [each.lower() for each in db.index]:
            # Get drug info
            drug_info, drug_info_found = get_values(db, drug, focus) 
            #print(drug_info)
            #results["results"].append(drug_info)    
            
        # if drug is in the "generic_name"
        if drug in db["generic_name"] and (not drug_info_found):
            drug_info, drug_info_found = get_values(db.set_index("generic_name"), drug, focus)
            #results["results"].append(drug_info) 
        # If drug is in the "brand_name"
        if drug in  db["brand_names"] and (not drug_info_found):
            drug_info, drug_info_found = get_values(db.set_index("brand_name"), drug, focus)
            #results["results"].append(drug_info) 
        # If drug is in the "related_drugs" category
        if drug in db["related_drugs"] and (not drug_info_found):
            drug_info, drug_info_found = get_values(db.set_index("related_drugs"), drug, focus)
            #results["results"].append(drug_info) 
        
    
        # If no drug was found at all then return None for all extracted feature attribute.
        if not drug_info_found and len(focus) > 0:
            drug_info = {"drug_name": drug}            
            for feature in set(focus):
                drug_info[feature] = None
#         else:
#             drug_info = {"drug_name": drug}
#             for feature in df.columns:
#                 drug_info[feature] = None
                
        results["results"].append(drug_info)

    return results
        
        
def get_values(df,drug,features):
    """
        This function gets feature values of the queried drug.
    """
    drug_info = {"drug_name": drug}
    #features = set(features)      
    
    if len(features) > 0:
        for feature in features:
            # Check if focus feature is in the database column
            if feature in df.columns:
                if feature == "generic_name":
                    drug_info[feature] = df.loc[drug, feature].split(",")[0]        
                elif feature == 'CSA':
                    drug_info[feature]= df.loc[drug, feature]
                elif feature == 'pregnancy_category':
                    drug_info[feature]= df.loc[drug, feature]
                else:
                    drug_info[feature] = df.loc[drug, feature].split(',')

    # Else just return all the attributes of the index drug
    else:
        for feature in db.columns:
            if feature == "generic_name":
                    drug_info[feature] = df.loc[drug, feature].split(",")[0]        
            elif feature == 'CSA':
                drug_info[feature]= df.loc[drug, feature]
            elif feature == 'pregnancy_category':
                drug_info[feature]= df.loc[drug, feature]
            else:
                drug_info[feature] = df.loc[drug, feature].split(',')

    return drug_info , True



In [131]:

# Load model from storage.
model, tokenizer = load_model_tokenizer("/kaggle/working/finetuned_NER.model", len(label_names))

In [132]:
query= "I want information on the drug doxycycline alongside its side effects . I want information on Amoxicillin too.\
Can you tell me about Tetracycline? Get some brand names and medical conditions while at it."

path = "/kaggle/working/drugs_info.parquet"
get_response(model,tokenizer,query, path, ["pregnancy_category", "generic_name"])

{'results': [{'drug_name': 'amoxicillin',
   'generic_name': 'amoxicillin',
   'brand_names': ['Amoxil', ' Trimox', ' Moxatag'],
   'pregnancy_category': 'B',
   'side_effects': ['severe stomach pain',
    ' or diarrhea that is watery or bloody even if it occurs months after your last dose. common amoxicillin side effects may include nausea ',
    ' vomiting ',
    ' diarrhea',
    ' or rash.'],
   'medical_condition': ['Bacterial Endocarditis Prophylaxis',
    '  Chlamydia Infection',
    '  Helicobacter pylori Infection',
    '  Lyme Disease - Arthritis',
    '  Lyme Disease - Carditis',
    '  Lyme Disease - Erythema Chronicum Migrans',
    '  Lyme Disease',
    '  Pneumonia',
    '  Bronchitis',
    '  Sinusitis',
    '  Urinary Tract Infection',
    '  Tonsillitis/Pharyngitis',
    '  Skin and Structure Infection',
    '  Cutaneous Bacillus anthracis',
    '  Anthrax Prophylaxis']},
  {'drug_name': 'tetracycline',
   'generic_name': 'tetracycline',
   'brand_names': ['Ala-Tet',
  

As you can see that the model performs really well in this example given. Let's see what happpens if we remove capital letters from the drugs. Will that affect the results?

I also introduce some false characters and delete some original letters from the name of the drugs. I also remove focus or features from the list forcing the model to figure those out on its own.

In [137]:
query= "I want information on the drug doxYCyclinE_ alongside its side effects . I want information on Amoxic@llin too.\
Can you tell me about tetracycli? Get some brand names and medical conditions while at it."

path = "/kaggle/working/drugs_info.parquet"
get_response(model,tokenizer,query, path, )

{'results': [{'drug_name': 'llin',
   'brand_names': None,
   'side_effects': None,
   'others': None,
   'drug': None,
   'medical_condition': None},
  {'drug_name': 'tetracycli',
   'brand_names': None,
   'side_effects': None,
   'others': None,
   'drug': None,
   'medical_condition': None},
  {'drug_name': 'doxycycline',
   'brand_names': ['Acticlate',
    ' Adoxa CK',
    ' Adoxa Pak',
    ' Adoxa TT',
    ' Alodox',
    ' Avidoxy',
    ' Doryx',
    ' Mondoxyne NL',
    ' Monodox',
    ' Morgidox',
    ' Okebo',
    ' Oracea',
    ' Oraxyl',
    ' Periostat Targadox',
    ' Vibramycin calcium',
    ' Vibramycin Hyclate',
    ' Vibramycin monohydrate',
    ' Vibra-Tabs'],
   'side_effects': ['hives',
    ' difficult breathing',
    ' swelling in your face or throat or a severe skin reaction fever',
    ' sore throat',
    ' burning in your eyes',
    ' skin pain',
    ' red or purple skin rash that spreads and causes blistering and peeling. seek medical treatment if you have a se

We see that the model was still able to identify tetracycline and doxycycline as drugs though it made so many errors for other parts. It was also able to identify specifically written focuses in the query.

Now, lets try to query with just one word: a drug name.

In [ ]:
query= "Doxycycline is my qa"

path = "/kaggle/working/drugs_info.parquet"
get_response(model,tokenizer,query, path)

## Challenges encountered during Project.
- There were some difficulty tesing code for deployment because of some dependency issues with the transformers library and the windows operating system.
- Unable to test model for deployment within allotted time.
- There was some time constraint as regards iimplementation of all steps required.

## Further Suggestions
- Model can be adapted to other medical tasks
- An autocorrect model can be created to correct spelling of users before they are fed to the model for results
- Model can be retrained on a larger bank of text to suit a bigger task.
- Model can be made more robust to handle 
- Retrieval function can be optimized and also adopted fully with just parquet files in streams.